## Mission 3: 패션 스타일 선호 여부 예측

#### 3-1. 추천 시스템의 기본인 협업 필터링 (Collaborative Filtering)은 크게 user-based filtering, item-based filtering 방식으로 나뉘어져 있다. 각각에 대해서 이해하고,2-2에서 구해 본 응답자의 “스타일 선호 정보표”를 토대로 Validation 데이터 내 응답자의“스타일 선호 여부 예측” 문제를 2가지 기법으로 어떻게 적용해 볼 수 있고, 서로 비교하여 어떤 장단점을 갖는지 설명한다.

##### ※ 설명을 용이하게 하기 위해 응답자의 스타일 선호도 예시를 들어서 설명해도 무방하다.



> 3-1에 대한 답변은 PPT 파일의 Page 61 ~ 82를 참고해주세요.



#### 3-1에서 살펴 본 기법 중, item-based filtering을 직접 구현해본다. “이미지 간 유사도”(image2image)만을 활용하여 Validation 데이터 내 응답자의 “스타일 선호 여부 예측”문제를 수행하고 성능을 측정한다.
##### ※ Hint. 1-2에서 학습한 ResNet-18의 중간 layer 값을 활용하여 각 이미지의 feature vector를 구하고, 벡터 연산을 통해 이미지 간 유사도를 구해볼 수 있다.
 ##### ※ 예측 문제에서 활용한 파라미터 및 임계 값 등의 수치를 정확하게 제시한다.



In [ ]:
import os
import json
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
import pandas as pd
from PIL import Image

In [ ]:
# 경로 설정
# /PATH/TO/ 부분을 심사 환경에 맞게 수정해주세요

training_image_path = '/PATH/TO/Dataset/bg_remove/training_image_no_bg' #training_image 를 불러오기 위한 경로 설정
validation_image_path = '/PATH/TO/Dataset/bg_remove/validation_image_no_bg' #validation_image 를 불러오기 위한 경로 설정
model_path = '/PATH/TO/Mission1/resnet18_gender_style_pretrained.pth'  # 1-2에서 학습된 모델 가중치 파일 경로
json_path = '/PATH/TO/Mission2/top_100_preference.json'  # 2-2에서 생성된 CSV 파일 경로

In [ ]:
# ResNet-18 모델 로드 및 학습된 가중치 불러오기
model = models.resnet18(pretrained=False)  # 학습된 가중치를 로드할 것이므로 pretrained=False 설정
model.fc = nn.Identity()  # 마지막 FC 레이어를 제거하여 특징 벡터를 추출하도록 설정
model.load_state_dict(torch.load(model_path), strict=False)  # strict=False로 불필요한 키 무시하고 가중치 불러오기
model.eval()  # 평가 모드로 전환

C:\Users\HappySnupy\PycharmProjects\DDC_2024\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\HappySnupy\PycharmProjects\DDC_2024\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
C:\Users\HappySnupy\AppData\Local\Temp\ipykernel_86484\3005752051.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future rel

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
# 이미지 전처리 설정
transform = transforms.Compose([
    transforms.Resize((224, 224)), #각기 다른 이미지의 크기를 224 * 224 사이즈로 모두 재설정
    transforms.ToTensor(), #각 이미지를 Tensor로 변환
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), #nomalize를 위한 값 설정
])

In [ ]:
# 특징 벡터 추출 함수 정의
def extract_feature_vector(image_path):
    image = Image.open(image_path).convert('RGB') #모든 이미지를 열고 RGB 형식으로 변환
    image = transform(image).unsqueeze(0)  # 이미지에 전처리 적용 후, 배치 차원을 추가

    # 모델을 통해 특징 벡터 추출 (학습 중이 아니므로 no_grad() 사용)
    with torch.no_grad():
        feature_vector = model(image).squeeze().numpy()  # 모델을 통해 나온 결과를 NumPy 배열로 변환하고 배치 차원 제거

    return feature_vector  # 추출된 특징 벡터 반환

In [ ]:
# JSON 파일을 읽기 위한 파일 열기 (utf-8 인코딩 사용)
with open(json_path, 'r', encoding='utf-8') as f:
    data = json.load(f)  # JSON 파일 내용을 딕셔너리 형태로 로드

# 로드한 데이터를 DataFrame 형식으로 변환 (분석 및 조작을 쉽게 하기 위함)
df = pd.DataFrame(data)

In [ ]:
# 사용자별 평균 벡터 계산 및 Validation 데이터 유사도 비교
results = []

# 총 사용자 수 계산
total_users = len(data)
user_counter = 0  # 사용자 진행 카운터 초기화

# json파일의 user id에 따른 각 사용자에 대해 반복문 실행
for user_id, user_data in data.items():
    user_counter += 1
    print(f"Processing user {user_counter}/{total_users} (ID: {user_id})...")  # 진행 현황 출력 (현재 유저/전체 유저  유저ID  확인한 이미지)

    # Training 데이터에서 선호 및 비선호 파일 리스트 추출
    training_preferred_files = user_data['Training']['선호']
    training_non_preferred_files = user_data['Training']['비선호']
    validation_preferred_files = user_data['Validation']['선호']
    validation_non_preferred_files = user_data['Validation']['비선호']

    # Training 데이터의 선호 및 비선호 파일의 특징 벡터를 추출하고, 평균 벡터 계산
    preferred_features = [extract_feature_vector(os.path.join(training_image_path, img)) for img in training_preferred_files if img]
    non_preferred_features = [extract_feature_vector(os.path.join(training_image_path, img)) for img in training_non_preferred_files if img]

    # 선호 파일이 있을 경우, 선호 파일의 평균 벡터 계산
    average_preferred_vector = np.mean(preferred_features, axis=0) if preferred_features else None
    # 비선호 파일이 있을 경우, 비선호 파일의 평균 벡터 계산
    average_non_preferred_vector = np.mean(non_preferred_features, axis=0) if non_preferred_features else None

    # Validation 데이터의 특징 벡터 추출 및 유사도 비교
    for val_img in validation_preferred_files:
        if val_img:  # 빈 문자열 또는 None 방지
            # Validation 데이터의 특징 벡터 추출
            val_feature = extract_feature_vector(os.path.join(validation_image_path, val_img))
            if average_preferred_vector is not None:
                # Validation 데이터의 특징 벡터와 평균 벡터 간 코사인 유사도 계산
                similarity = cosine_similarity(val_feature.reshape(1, -1), average_preferred_vector.reshape(1, -1))[0][0]
                # 유사도를 계속 수정하며 코드 진행
                predicted_label = 1 if similarity >= 0.85 else 0
                # 실제 레이블은 선호
                true_label = 1
                # 예측 결과와 실제 레이블 비교
                result = 'O' if true_label == predicted_label else 'X' # 맞췄다면 O, 틀렸다면 X

                # 결과 추가
                results.append({
                    'user_id': user_id,
                    'validation_file': val_img,
                    'true_label': true_label,
                    'predicted_label': predicted_label,
                    'result': result,
                    'preference_type': '선호'
                })

                # 진행 현황 출력
                print(f"Validation (선호): {val_img}, Similarity: {similarity:.2f}, Predicted: {'선호' if predicted_label == 1 else '비선호'}, Result: {result}")

    for val_img in validation_non_preferred_files:
        if val_img:  # 빈 문자열 또는 None 방지
            # Validation 데이터의 특징 벡터 추출
            val_feature = extract_feature_vector(os.path.join(validation_image_path, val_img))
            if average_non_preferred_vector is not None:
                # Validation 데이터의 특징 벡터와 평균 벡터 간 코사인 유사도 계산
                similarity = cosine_similarity(val_feature.reshape(1, -1), average_non_preferred_vector.reshape(1, -1))[0][0]
                # 유사도를 계속 수정하며 코드 진행
                predicted_label = 1 if similarity >= 0.85 else 0
                # 실제 레이블은 비선호
                true_label = 0
                # 예측 결과와 실제 레이블 비교
                result = 'O' if true_label == predicted_label else 'X' # 맞췄다면 O, 틀렸다면 X

                # 결과 추가
                results.append({
                    'user_id': user_id,
                    'validation_file': val_img,
                    'true_label': true_label,
                    'predicted_label': predicted_label,
                    'result': result,
                    'preference_type': '비선호'
                })

                # 진행 현황 출력
                print(f"Validation (비선호): {val_img}, Similarity: {similarity:.2f}, Predicted: {'선호' if predicted_label == 1 else '비선호'}, Result: {result}")

print("Processing completed for all users.")  # 전체 처리 완료 메시지 출력

Processing user 1/100 (ID: 64747)...
Validation (선호): W_05628_00_cityglam_W.jpg, Similarity: 0.88, Predicted: 선호, Result: O
Validation (선호): W_20598_70_military_W.jpg, Similarity: 0.97, Predicted: 선호, Result: O
Validation (선호): W_22510_80_powersuit_W.jpg, Similarity: 0.85, Predicted: 선호, Result: O
Validation (선호): W_30988_90_kitsch_W.jpg, Similarity: 0.94, Predicted: 선호, Result: O
Validation (선호): W_37491_70_military_W.jpg, Similarity: 0.97, Predicted: 선호, Result: O
Validation (선호): W_38588_19_genderless_W.jpg, Similarity: 0.92, Predicted: 선호, Result: O
Validation (선호): W_39164_00_oriental_W.jpg, Similarity: 0.96, Predicted: 선호, Result: O
Validation (선호): W_44330_10_sportivecasual_W.jpg, Similarity: 0.94, Predicted: 선호, Result: O
Validation (선호): W_46907_80_powersuit_W.jpg, Similarity: 0.91, Predicted: 선호, Result: O
Validation (비선호): W_02498_50_feminine_W.jpg, Similarity: 0.92, Predicted: 선호, Result: X
Validation (비선호): W_11610_90_grunge_W.jpg, Similarity: 0.87, Predicted: 선호, Result: 

In [ ]:
# 모든 행과 열을 출력하도록 설정
pd.set_option('display.max_rows', None)  # 모든 행 출력
pd.set_option('display.max_columns', None)  # 모든 열 출력
pd.set_option('display.width', None)  # 출력의 너비를 화면에 맞추기
pd.set_option('display.max_colwidth', None)  # 각 열의 최대 너비를 None으로 설정

In [ ]:
# 6. 예측 결과 리스트 사용
results_list = results  # `results`는 이미 리스트 형태이므로 그대로 사용

In [ ]:
# 사용자별 예측 결과를 JSON 형식으로 정리
organized_results = {}

# 3-2의 예측 결과를 각 사용자별로 정리
for result in results_list:
    # 각 항목에 필요한 키가 모두 있는지 확인
    required_keys = ['user_id', 'validation_file', 'predicted_label', 'result', 'preference_type']
    if not all(key in result for key in required_keys):
        # 필요한 키가 누락된 경우 오류 발생
        raise KeyError(f"One of the required keys is missing in the result: {result}")

    user_id = result['user_id']  # 사용자 ID 추출

    # 현재 사용자 ID가 organized_results 에 없으면 새로운 항목 추가
    if user_id not in organized_results:
        organized_results[user_id] = {
            "Validation": {
                "선호": [],
                "비선호": []
            }
        }

    # 결과 항목 구성
    result_entry = {
        "파일명": result['validation_file'],  # 검증 파일명
        "예측 결과": "선호" if result['predicted_label'] == 1 else "비선호",  # 예측된 결과
        "결과": result['result']  # 예측의 맞고 틀림 여부 ('O' 또는 'X')
    }

    # 선호 또는 비선호 결과에 따라 정리된 결과에 추가
    if result['preference_type'] == "선호":
        organized_results[user_id]["Validation"]["선호"].append(result_entry)
    elif result['preference_type'] == "비선호":
        organized_results[user_id]["Validation"]["비선호"].append(result_entry)

In [ ]:
# JSON 파일로 저장 (정리된 데이터를 저장)
json_output_path = r'C:\Users\HappySnupy\OneDrive\바탕 화면\대학\활동\2024 데이터 크리에이터 캠프\2024 데이터 크리에이터 캠프 대학부 데이터셋\organized_results.json'
with open(json_output_path, 'w', encoding='utf-8') as f:
    json.dump(organized_results, f, ensure_ascii=False, indent=4)
print("Organized prediction results saved to JSON.")

Organized prediction results saved to JSON.


In [ ]:
# 사용자별 예측 결과 출력
top_100_users = list(organized_results.keys())[:100]  # 상위 100명의 사용자 ID를 추출
organized_output = []  # 출력용 리스트 초기화

# 각 사용자별로 예측 결과를 정리하여 출력용 리스트에 추가
for user_id in top_100_users:
    # Validation 데이터 중 선호 항목에 대한 결과를 문자열 형식으로 정리
    validation_preferred_results = [
        f"{result['파일명']} (예측: {result['예측 결과']}, 결과: {result['결과']})"
        for result in organized_results[user_id]["Validation"]["선호"]
    ]
    validation_preferred = '\n'.join(validation_preferred_results)  # 선호 결과를 줄바꿈하여 하나의 문자열로 변환

    # Validation 데이터 중 비선호 항목에 대한 결과를 문자열 형식으로 정리
    validation_non_preferred_results = [
        f"{result['파일명']} (예측: {result['예측 결과']}, 결과: {result['결과']})"
        for result in organized_results[user_id]["Validation"]["비선호"]
    ]
    validation_non_preferred = '\n'.join(validation_non_preferred_results)  # 비선호 결과를 줄바꿈하여 하나의 문자열로 변환

    # 응답자별 데이터를 리스트에 추가
    organized_output.append([
        user_id,  # 사용자 ID
        validation_preferred,  # 선호 파일에 대한 결과
        validation_non_preferred  # 비선호 파일에 대한 결과
    ])

In [ ]:
# Pandas 데이터프레임 생성 및 출력
# organized_output 리스트를 사용하여 데이터프레임 생성
organized_df = pd.DataFrame(organized_output, columns=[
    '응답자 ID',  # 사용자 ID 열
    'Validation 선호 파일 예측 결과',  # 선호 파일에 대한 예측 결과 열
    'Validation 비선호 파일 예측 결과'  # 비선호 파일에 대한 예측 결과 열
])
# 생성된 데이터프레임을 organized_df에 저장

In [ ]:
# 인덱스를 1부터 시작하게 설정
organized_df.index = pd.RangeIndex(start=1, stop=len(organized_df) + 1, step=1)  # 데이터프레임의 인덱스를 1부터 시작하도록 설정

# 데이터프레임 출력
print("----- Organized Prediction Results for Top 100 Users -----")  # 제목 출력
organized_df.head(100)  # 상위 100명의 예측 결과 데이터프레임을 출력

----- Organized Prediction Results for Top 100 Users -----


,응답자 ID,Validation 선호 파일 예측 결과,Validation 비선호 파일 예측 결과
1,64747,"W_05628_00_cityglam_W.jpg (예측: 선호, 결과: O)\nW_20598_70_military_W.jpg (예측: 선호, 결과: O)\nW_22510_80_powersuit_W.jpg (예측: 선호, 결과: O)\nW_30988_90_kitsch_W.jpg (예측: 선호, 결과: O)\nW_37491_70_military_W.jpg (예측: 선호, 결과: O)\nW_38588_19_genderless_W.jpg (예측: 선호, 결과: O)\nW_39164_00_oriental_W.jpg (예측: 선호, 결과: O)\nW_44330_10_sportivecasual_W.jpg (예측: 선호, 결과: O)\nW_46907_80_powersuit_W.jpg (예측: 선호, 결과: O)","W_02498_50_feminine_W.jpg (예측: 선호, 결과: X)\nW_11610_90_grunge_W.jpg (예측: 선호, 결과: X)\nW_14102_50_feminine_W.jpg (예측: 선호, 결과: X)\nW_27828_60_minimal_W.jpg (예측: 선호, 결과: X)\nW_34024_10_sportivecasual_W.jpg (예측: 선호, 결과: X)\nW_47169_70_hippie_W.jpg (예측: 선호, 결과: X)"
2,63405,"W_01853_60_mods_M.jpg (예측: 선호, 결과: O)\nW_02677_60_mods_M.jpg (예측: 선호, 결과: O)\nW_02879_90_hiphop_M.jpg (예측: 선호, 결과: O)\nW_04522_50_ivy_M.jpg (예측: 비선호, 결과: X)\nW_04684_90_hiphop_M.jpg (예측: 선호, 결과: O)\nW_06860_19_normcore_M.jpg (예측: 선호, 결과: O)\nW_15294_50_ivy_M.jpg (예측: 선호, 결과: O)","W_07187_70_hippie_M.jpg (예측: 선호, 결과: X)\nW_12304_80_bold_M.jpg (예측: 선호, 결과: X)\nW_12904_50_ivy_M.jpg (예측: 선호, 결과: X)\nW_15140_80_bold_M.jpg (예측: 비선호, 결과: O)\nW_16501_70_hippie_M.jpg (예측: 선호, 결과: X)\nW_16755_00_metrosexual_M.jpg (예측: 비선호, 결과: O)\nW_17443_90_hiphop_M.jpg (예측: 비선호, 결과: O)"
3,64346,"W_07316_00_metrosexual_M.jpg (예측: 선호, 결과: O)\nW_09154_50_ivy_M.jpg (예측: 선호, 결과: O)\nW_24103_50_ivy_M.jpg (예측: 비선호, 결과: X)\nW_29918_19_normcore_M.jpg (예측: 선호, 결과: O)\nW_29990_90_hiphop_M.jpg (예측: 선호, 결과: O)","W_00496_60_mods_M.jpg (예측: 선호, 결과: X)\nW_16121_80_bold_M.jpg (예측: 선호, 결과: X)\nW_16430_90_hiphop_M.jpg (예측: 선호, 결과: X)\nW_24250_90_hiphop_M.jpg (예측: 선호, 결과: X)\nW_24838_70_hippie_M.jpg (예측: 선호, 결과: X)\nW_24931_50_ivy_M.jpg (예측: 선호, 결과: X)\nW_26099_19_normcore_M.jpg (예측: 선호, 결과: X)"
4,64561,"W_06046_10_sportivecasual_W.jpg (예측: 선호, 결과: O)\nW_18205_50_feminine_W.jpg (예측: 선호, 결과: O)\nW_22239_60_space_W.jpg (예측: 선호, 결과: O)\nW_30671_70_hippie_W.jpg (예측: 선호, 결과: O)\nW_33305_60_space_W.jpg (예측: 선호, 결과: O)\nW_35091_80_powersuit_W.jpg (예측: 선호, 결과: O)\nW_38656_10_sportivecasual_W.jpg (예측: 선호, 결과: O)\nW_41448_10_sportivecasual_W.jpg (예측: 선호, 결과: O)","W_22943_10_athleisure_W.jpg (예측: 선호, 결과: X)\nW_23519_60_minimal_W.jpg (예측: 선호, 결과: X)\nW_33240_80_bodyconscious_W.jpg (예측: 선호, 결과: X)\nW_48457_60_minimal_W.jpg (예측: 선호, 결과: X)"
5,65139,"W_63644_10_sportivecasual_M.jpg (예측: 선호, 결과: O)","W_24517_70_hippie_M.jpg (예측: 선호, 결과: X)\nW_24717_60_mods_M.jpg (예측: 선호, 결과: X)\nW_27138_60_mods_M.jpg (예측: 선호, 결과: X)\nW_28314_10_sportivecasual_M.jpg (예측: 선호, 결과: X)\nW_29942_50_ivy_M.jpg (예측: 선호, 결과: X)\nW_31913_90_hiphop_M.jpg (예측: 선호, 결과: X)\nW_51514_50_ivy_M.jpg (예측: 비선호, 결과: O)\nW_52693_00_metrosexual_M.jpg (예측: 선호, 결과: X)\nW_54129_19_normcore_M.jpg (예측: 선호, 결과: X)\nW_54465_80_bold_M.jpg (예측: 선호, 결과: X)\nW_58793_00_metrosexual_M.jpg (예측: 선호, 결과: X)"
6,66513,"W_14828_50_classic_W.jpg (예측: 선호, 결과: O)","T_06910_50_classic_W.jpg (예측: 선호, 결과: X)\nW_10984_50_feminine_W.jpg (예측: 선호, 결과: X)\nW_14914_50_feminine_W.jpg (예측: 선호, 결과: X)\nW_37404_60_space_W.jpg (예측: 선호, 결과: X)\nW_39793_80_powersuit_W.jpg (예측: 선호, 결과: X)\nW_44520_70_punk_W.jpg (예측: 선호, 결과: X)\nW_53112_90_lingerie_W.jpg (예측: 선호, 결과: X)\nW_56334_10_sportivecasual_W.jpg (예측: 비선호, 결과: O)\nW_60553_00_cityglam_W.jpg (예측: 선호, 결과: X)"
7,59704,"W_01549_50_ivy_M.jpg (예측: 선호, 결과: O)\nW_01853_60_mods_M.jpg (예측: 선호, 결과: O)\nW_02728_60_mods_M.jpg (예측: 선호, 결과: O)\nW_04636_50_ivy_M.jpg (예측: 선호, 결과: O)\nW_12092_80_bold_M.jpg (예측: 선호, 결과: O)\nW_15244_80_bold_M.jpg (예측: 선호, 결과: O)\nW_16219_70_hippie_M.jpg (예측: 비선호, 결과: X)","W_06875_90_hiphop_M.jpg (예측: 선호, 결과: X)\nW_12476_90_hiphop_M.jpg (예측: 선호, 결과: X)\nW_15120_60_mods_M.jpg (예측: 선호, 결과: X)\nW_17697_50_ivy_M.jpg (예측: 선호, 결과: X)\nW_19833_50_ivy_M.jpg (예측: 비선호, 결과: O)"
8,60173,"W_00152_50_feminine_W.jpg (예측: 선호, 결과: O)\nW_06015_80_powersuit_W.jpg (예측: 선호, 결과: O)\nW_14570_60_minimal_W.jpg (예측: 선호, 결과: O)","W_00351_70_hippie_W.jpg (예측: 선호, 결과: X)\nW_01236_10_sportivecasual_W.jpg (예측: 비선호, 

In [ ]:
# 예측 결과를 데이터프레임으로 변환
results_df = pd.DataFrame(results)

# 성능 평가
true_labels = results_df['true_label']
predicted_labels = results_df['predicted_label']

# 정확도, 정밀도, 재현율, F1 점수 계산
accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels)
recall = recall_score(true_labels, predicted_labels)
f1 = f1_score(true_labels, predicted_labels)

# 성능 결과 출력
print("----- Model Performance Metrics -----")
print(f"Accuracy (정확도): {accuracy:.2f}")
print(f"Precision (정밀도): {precision:.2f}")
print(f"Recall (재현율): {recall:.2f}")
print(f"F1 Score: {f1:.2f}")
print("-------------------------------------")

----- Model Performance Metrics -----
Accuracy (정확도): 0.42
Precision (정밀도): 0.40
Recall (재현율): 0.92
F1 Score: 0.56
-------------------------------------
